# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that I will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json

## Experiment Environment and Compute Cluster

Get Experiment name and identify the compute cluster. In the cell below, the code accesses the data that I use in this project.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Heart-Failure-Project'

experiment=Experiment(ws, experiment_name)

#Check if the cluster exists if it does not then create one.
# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-11-14T08:36:43.296000+00:00', 'errors': None, 'creationTime': '2020-11-14T08:36:39.428933+00:00', 'modifiedTime': '2020-11-14T08:36:55.297086+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

Set up the Hyperdrive configuration:</p>
1.BanditPolicy is used as a termination policy which every two iterations checks if the primary metric which is the accuracy falls outside the top 10% range.</p>
2.Logistic Regression with the train.py file is used and the parameters to be improved are C and max_iter. 
<p> 
3.Estimator is specified using SKLEARN and the hyperdrive run config is also specified using the estimator, termination policy and the parameters to be improved.

In [8]:
# Create a bandit termination policy. 
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

#Specify parameter sampler 
ps = RandomParameterSampling( {
        "C": uniform(0.1, 1),
        "max_iter": choice(5, 10,  20,  40, 50, 80, 100, 150, 200),
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#Create an SKLEARN estimator and hyperdrive config
est = SKLearn(source_directory='.',
             compute_target=compute_target,
             entry_script='train.py'
             )

hyperdrive_run_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=early_termination_policy, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [9]:
#Submit experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


## Run Details

Run `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

...

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

Get the best model from the hyperdrive experiments and diplay all the properties of the model.

In [12]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

['--C', '0.26880111988221433', '--max_iter', '40']
['azureml-logs/55_azureml-execution-tvmps_9a4b82a0ac301d7d9260c6ffca8b1864c789b75c60b3bad6826130985f99754e_d.txt', 'azureml-logs/65_job_prep-tvmps_9a4b82a0ac301d7d9260c6ffca8b1864c789b75c60b3bad6826130985f99754e_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_9a4b82a0ac301d7d9260c6ffca8b1864c789b75c60b3bad6826130985f99754e_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/103_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [13]:
#Save the best model
model = best_run.register_model(model_name='sklearn-train-model', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service